In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [5]:
print(keras.__version__)

2.15.0


In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [7]:
X_train = x_train.reshape(-1,28, 28, 1)  # reshaping for convnet
X_test = x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [8]:
print(X_train.shape)

(60000, 28, 28, 1)


In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

(48000, 28, 28, 1)


In [10]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

In [11]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128, 256]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


In [12]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')


my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)


Trial 10 Complete [00h 01m 26s]
accuracy: 0.9703333377838135

Best accuracy So Far: 0.9834166765213013
Total elapsed time: 00h 11m 43s


In [13]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 26, 26, 102)       1020      
                                                                 
 conv2d_66 (Conv2D)          (None, 24, 24, 42)        38598     
                                                                 
 conv2d_67 (Conv2D)          (None, 22, 22, 67)        25393     
                                                                 
 conv2d_68 (Conv2D)          (None, 20, 20, 37)        22348     
                                                                 
 conv2d_69 (Conv2D)          (None, 18, 18, 52)        17368     
                                                                 
 flatten_11 (Flatten)        (None, 16848)             0         
                                                                 
 dense_16 (Dense)            (None, 256)             

In [14]:
path = '/content/drive/MyDrive/mlops/model'
model.save(path)

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)


{'serving_default': {'inputs': ['inputs'], 'outputs': ['outputs']}}


In [16]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

In [17]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)